text.replace("\\", "/")

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
spark

In [6]:
parent_path = 'D:/skolaÅr5/MasterThesis/mimic-iv-1.0/'

In [7]:
codes_icd_path = parent_path + '/hosp/diagnoses_icd.csv'
diagnose_codes_icd = spark.read.load(codes_icd_path, format="csv", sep=",", inferSchema="true", header="true")

In [8]:
diagnoses_codes_path = parent_path + '/hosp/d_icd_diagnoses.csv'
diagnoses_codes = spark.read.load(diagnoses_codes_path,  format="csv", sep=",", inferSchema="true", header="true")  

In [9]:
patients_path = parent_path + '/core/patients.csv'
patients = spark.read.load(patients_path,  format="csv", sep=",", inferSchema="true", header="true")  

In [10]:
admission_path = parent_path + '/core/admissions.csv'
admissions = spark.read.load(admission_path,  format="csv", sep=",", inferSchema="true", header="true")  

In [11]:
admissions_ = admissions.withColumnRenamed('subject_id', 'subject_id2')
patients_admissions = patients.join(admissions_, patients.subject_id == admissions_.subject_id2, 'inner').drop('subject_id2')

In [12]:
from pyspark.sql.functions import udf,col

In [13]:
diagnoses_codes_ = diagnoses_codes.withColumnRenamed('icd_code', 'code').withColumnRenamed('icd_version', 'version')

patients_diagnoses = diagnose_codes_icd.join(diagnoses_codes_, (diagnose_codes_icd.icd_code == diagnoses_codes_.code) & (diagnose_codes_icd.icd_version == diagnoses_codes_.version), "inner").drop('code', 'version')

In [14]:
patients_diagnoses = patients_diagnoses.withColumnRenamed('subject_id', 'sub2').withColumnRenamed('hadm_id', 'h_id')

In [15]:
final_data = patients_admissions.join(patients_diagnoses, ((patients_admissions.subject_id == patients_diagnoses.sub2) & (patients_admissions.hadm_id == patients_diagnoses.h_id)) , 'inner').drop('sub2', 'h_id')

In [16]:
final_data.show()

+----------+------+----------+-----------+-----------------+----------+--------+-------------------+-------------------+-------------------+-----------------+--------------------+------------------+---------+--------+--------------+---------+-------------------+-------------------+--------------------+-------+--------+-----------+--------------------+
|subject_id|gender|anchor_age|anchor_year|anchor_year_group|       dod| hadm_id|          admittime|          dischtime|          deathtime|   admission_type|  admission_location|discharge_location|insurance|language|marital_status|ethnicity|          edregtime|          edouttime|hospital_expire_flag|seq_num|icd_code|icd_version|          long_title|
+----------+------+----------+-----------+-----------------+----------+--------+-------------------+-------------------+-------------------+-----------------+--------------------+------------------+---------+--------+--------------+---------+-------------------+-------------------+----------

In [17]:
needed_columns = set(['subject_id', 'age', 'hadm_id', 'icd_code', 'icd_version', 'long_title'])

In [18]:
columns_ = [c for c in final_data.columns if c not in needed_columns]

In [19]:
columns_

['gender',
 'anchor_age',
 'anchor_year',
 'anchor_year_group',
 'dod',
 'admittime',
 'dischtime',
 'deathtime',
 'admission_type',
 'admission_location',
 'discharge_location',
 'insurance',
 'language',
 'marital_status',
 'ethnicity',
 'edregtime',
 'edouttime',
 'hospital_expire_flag',
 'seq_num']

In [20]:
new = final_data.drop(*columns_).na.drop()

In [3]:
new.show()

NameError: name 'new' is not defined

In [10]:
import dask.dataframe as dd

In [28]:
df = dd.read_csv('D:/skolaÅr5/MasterThesis/mimic-iv-1.0/wooo.csv', dtype={'hadm_id': 'object',
       'icd_version': 'object',
       'subject_id': 'object'})

In [29]:
df.columns

Index(['subject_id', 'hadm_id', 'icd_code', 'icd_version', 'long_title'], dtype='object')

In [30]:
df.head()

,subject_id,hadm_id,icd_code,icd_version,long_title
0,10003637,23487925,I2582,10,Chronic total occlusion of coronary artery
1,10003637,23487925,I252,10,Old myocardial infarction
2,10003637,23487925,K611,10,Rectal abscess
3,10003637,23487925,I2510,10,Atherosclerotic heart disease of native corona...
4,10003637,23487925,I509,10,"Heart failure, unspecified"
